In [23]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

# ML Model Creation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer

import statsmodels.api as sm

current_dir = os.getcwd()
data_fldr = os.path.join(current_dir, "data")
out_dir = os.path.join(current_dir, "data", "output")

# Datasets

In [2]:
coe_df = pd.read_excel(os.path.join(data_fldr, "COE_Export.xlsx"), sheet_name="Yearly")
cpi_df = pd.read_excel(os.path.join(data_fldr, "ConsumerPriceIndex.xlsx"), sheet_name="Consolidate")
ni_df = pd.read_excel(os.path.join(data_fldr, "NationalIncome.xlsx"), sheet_name="Consolidate")
hh_df = pd.read_excel(os.path.join(data_fldr, "Household.xlsx"), sheet_name="Consolidate")
ms_df = pd.read_excel(os.path.join(data_fldr, "MaritalStatus.xlsx"), sheet_name="Consolidate")
pp_df = pd.read_excel(os.path.join(data_fldr, "Population.xlsx"), sheet_name="Consolidate")


In [3]:
# coe_df
# cpi_df

## Choosing of COE Category

In [4]:
coe_cat_df = coe_df.loc[coe_df['Category'] == "A", :]
coe_cat_df

,Year,Category,Value
0,2000,A,38981.083333
5,2001,A,27031.791667
10,2002,A,30831.916667
15,2003,A,28754.875000
20,2004,A,25180.916667
25,2005,A,16550.791667
30,2006,A,11187.166667
35,2007,A,14101.125000
40,2008,A,12330.291667
45,2009,A,11600.041667


## Joining of Datasets

### Full Dataframe

In [5]:
# List of DataFrames
dfs = [coe_cat_df, cpi_df, ni_df, hh_df, ms_df, pp_df]

# Merge DataFrames using functools.reduce()
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Year', how='left'), dfs)
merged_df

,Year,Category,Value,MAS Core Inflation Measure (Index),Residential Properties Price Index,Landed Price Index,Non-Landed Price Index,Per Capita GDP (Dollar),HDB 1- And 2-Room Flats,HDB 3-Room Flats,...,Total Female Malays,Total Chinese,Total Male Chinese,Total Female Chinese,Total Indians,Total Male Indians,Total Female Indians,Other Ethnic Groups (Total),Other Ethnic Groups (Males),Other Ethnic Groups (Females)
0,2000,A,38981.083333,73.818,97.950,99.350,97.700,41121.0,45600.0,235700.0,...,226357.0,2513847.0,1249662.0,1264185.0,257866.0,134337.0,123529.0,46443.0,21818.0,24625.0
1,2001,A,27031.791667,74.965,88.200,90.575,87.450,38880.0,41800.0,233400.0,...,229818.0,2552077.0,1267019.0,1285058.0,262968.0,136485.0,126483.0,49069.0,23084.0,25985.0
2,2002,A,30831.916667,74.927,82.475,85.225,81.575,39679.0,41400.0,225300.0,...,233284.0,2589525.0,1283362.0,1306163.0,271923.0,140768.0,131155.0,53136.0,25089.0,28047.0
3,2003,A,28754.875000,75.631,81.000,83.625,80.150,41343.0,44200.0,225800.0,...,234194.0,2572607.0,1273256.0,1299351.0,269899.0,138642.0,131257.0,54594.0,25906.0,28688.0
4,2004,A,25180.916667,77.135,80.675,83.150,79.975,46664.0,41200.0,224700.0,...,237453.0,2599813.0,1285557.0,1314256.0,278106.0,142754.0,135352.0,59658.0,28484.0,31174.0
5,2005,A,16550.791667,78.104,83.000,84.625,82.700,49867.0,44500.0,211600.0,...,240316.0,2626723.0,1297849.0,1328874.0,291131.0,149621.0,141510.0,69238.0,33263.0,35975.0
6,2006,A,11187.166667,79.445,88.950,88.875,89.175,53655.0,46200.0,229300.0,...,243219.0,2656358.0,1310873.0,1345485.0,303096.0,155938.0,147158.0,80462.0,38672.0,41790.0
7,2007,A,14101.125000,81.195,109.925,103.850,111.575,59429.0,44700.0,221300.0,...,245583.0,2686997.0,1324715.0,1362282.0,313395.0,161484.0,151911.0,92138.0,44309.0,47829.0
8,2008,A,12330.291667,85.802,123.475,115.175,125.400,56607.0,43000.0,223200.0,...,248093.0,2721779.0,1339596.0,1382183.0,323431.0,167149.0,156282.0,102339.0,49230.0,53109.0
9,2009,A,11600.041667,85.783,106.000,105.850,106.050,56619.0,49700.0,226500.0,...,250936.0,2770303.0,1360224.0,1410079.0,343509.0,178129.0,165380.0,120013.0,57264.0,62749.0


### Only race

In [41]:
coe_pp = pd.merge(left=coe_cat_df, right=pp_df, left_on="Year", right_on="Year", how="left")
coe_pp_drop = coe_pp.drop(['Year','Category',], axis=1).drop([24], axis=0)
# coe_pp_drop = coe_pp_drop.iloc[:, [0, 4,7,10,13]]
coe_pp_drop

coe_pp_drop.corr()

,Value,Total Residents,Total Male Residents,Total Female Residents,Total Malays,Total Male Malays,Total Female Malays,Total Chinese,Total Male Chinese,Total Female Chinese,Total Indians,Total Male Indians,Total Female Indians,Other Ethnic Groups (Total),Other Ethnic Groups (Males),Other Ethnic Groups (Females)
Value,1.000000,0.594276,0.589969,0.597601,0.588093,0.589069,0.586739,0.604469,0.601948,0.606334,0.545350,0.536904,0.552941,0.544404,0.518953,0.563420
Total Residents,0.594276,1.000000,0.999842,0.999898,0.985694,0.980716,0.989387,0.996786,0.996754,0.996764,0.975236,0.967859,0.981034,0.958719,0.936813,0.972997
Total Male Residents,0.589969,0.999842,1.000000,0.999487,0.983272,0.977948,0.987278,0.995394,0.995426,0.995324,0.978561,0.971821,0.983713,0.963080,0.942199,0.976480
Total Female Residents,0.597601,0.999898,0.999487,1.000000,0.987415,0.982716,0.990856,0.997676,0.997593,0.997693,0.972342,0.964457,0.978658,0.954997,0.932273,0.969977
Total Malays,0.588093,0.985694,0.983272,0.987415,1.000000,0.999529,0.999618,0.991923,0.991697,0.992046,0.930423,0.918669,0.940701,0.905339,0.873501,0.928169
Total Male Malays,0.589069,0.980716,0.977948,0.982716,0.999529,1.000000,0.998299,0.988784,0.988561,0.988906,0.919772,0.907571,0.930518,0.893655,0.859962,0.918091
Total Female Malays,0.586739,0.989387,0.987278,0.990856,0.999618,0.998299,1.000000,0.993952,0.993725,0.994077,0.939275,0.927932,0.949123,0.915141,0.885003,0.936506
Total Chinese,0.604469,0.996786,0.995394,0.997676,0.991923,0.988784,0.993952,1.000000,0.999965,0.999980,0.954863,0.945172,0.963024,0.933450,0.906659,0.951930
Total Male Chinese,0.601948,0.996754,0.995426,0.997593,0.991697,0.988561,0.993725,0.999965,1.000000,0.999893,0.954944,0.945369,0.962987,0.933487,0.906708,0.951958
Total Female Chinese,0.606334,0.996764,0.995324,0.997693,0.992046,0.988906,0.994077,0.999980,0.999893,1.000000,0.954758,0.944981,0.963007,0.933379,0.906581,0.951864


# Machine Learning Portion

In [42]:
def log_scale(X):
    return np.log1p(X)

In [43]:
# Initialize FunctionTransformer
transformer = FunctionTransformer(log_scale)

In [44]:
# Step 4: Split your data into training and testing sets
X = coe_pp_drop.drop('Value', axis=1)  # Features
y = coe_pp_drop['Value']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_log_scaled = transformer.transform(X_train)
# Add a constant to the X_train_log_scaled for the intercept term
X_train_log_scaled_with_const = sm.add_constant(X_train_log_scaled)

# Step 5: Initialize and train your multilinear regression model
model = LinearRegression()
# Fit the linear regression model using statsmodels
model.fit(X_train, y_train)
model_stats = sm.OLS(y_train, X_train_log_scaled_with_const).fit()

# Step 6: Evaluate the model on the test set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 275999705.4379314


c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [45]:
# Print the summary to see the p-values
print(model_stats.summary())

                            OLS Regression Results                            
Dep. Variable:                  Value   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     22.89
Date:                Fri, 07 Jun 2024   Prob (F-statistic):             0.0126
Time:                        22:40:37   Log-Likelihood:                -173.46
No. Observations:                  19   AIC:                             378.9
Df Residuals:                       3   BIC:                             394.0
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [19]:
# Step 7: Analyze the coefficients to determine the importance of factors
coefficients = pd.DataFrame({'feature': X.columns, 'coefficient': model.coef_})
coefficients = coefficients.sort_values(by='coefficient', ascending=False)
print("Coefficients:")
print(coefficients)

Coefficients:
                          feature   coefficient
3                    Total Malays  1.453297e+10
6                   Total Chinese  7.170143e+09
2          Total Female Residents  5.669941e+09
1            Total Male Residents  5.379868e+09
12    Other Ethnic Groups (Total)  1.857987e+08
10             Total Male Indians  1.022686e+07
11           Total Female Indians  2.492745e+06
9                   Total Indians -1.255931e+07
13    Other Ethnic Groups (Males) -8.468358e+07
14  Other Ethnic Groups (Females) -1.021387e+08
7              Total Male Chinese -3.457609e+09
8            Total Female Chinese -3.732410e+09
4               Total Male Malays -7.240233e+09
5             Total Female Malays -7.292311e+09
0                 Total Residents -1.102453e+10
